<a href="https://colab.research.google.com/github/frvalov3/SQLInjection/blob/main/SQLInjection_using_ML_vs_sqli_data_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
cd /content

/content


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Pull Input data files from my github to colab
!git clone https://github.com/frvalov3/SQLInjection



# Any results you write to the current directory are saved as output.

fatal: destination path 'SQLInjection' already exists and is not an empty directory.


In [2]:
pwd

'/content'

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras import models, layers
import warnings

warnings.filterwarnings('ignore')

# Reading Data

In [5]:
import pandas as pd
df = pd.read_csv("/content/SQLInjection/InputData/InputData/sqli.csv",encoding='utf-16')

In [6]:
df.head()

,Sentence,Label
0,a,1
1,a',1
2,a' --,1
3,a' or 1 = 1; --,1
4,@,1


In [7]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:

X = df['Sentence']
y = df['Label']
vectorizer = CountVectorizer(min_df = 2, max_df = 0.8, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(X.values.astype('U')).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)

# Training data in many model

***Using Logistic Regression***

In [12]:
from sklearn.metrics import accuracy_score
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9273809523809524


**Using simple Neuron Network**

In [13]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer

In [14]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [15]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                94360     
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 1024)              11264     
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1025      
                                                        

In [16]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=15)

Epoch 1/10
224/224 [==============================] - 3s 7ms/step - loss: 0.2173 - accuracy: 0.8991 - val_loss: 0.6900 - val_accuracy: 0.7298
Epoch 2/10
224/224 [==============================] - 1s 6ms/step - loss: 0.1103 - accuracy: 0.9512 - val_loss: 0.3048 - val_accuracy: 0.8048
Epoch 3/10
224/224 [==============================] - 1s 6ms/step - loss: 0.0923 - accuracy: 0.9658 - val_loss: 0.1343 - val_accuracy: 0.9226
Epoch 4/10
224/224 [==============================] - 1s 6ms/step - loss: 0.0795 - accuracy: 0.9702 - val_loss: 0.0724 - val_accuracy: 0.9774
Epoch 5/10
224/224 [==============================] - 1s 6ms/step - loss: 0.0877 - accuracy: 0.9690 - val_loss: 0.0598 - val_accuracy: 0.9821
Epoch 6/10
224/224 [==============================] - 1s 6ms/step - loss: 0.0819 - accuracy: 0.9720 - val_loss: 0.0677 - val_accuracy: 0.9810
Epoch 7/10
224/224 [==============================] - 1s 6ms/step - loss: 0.0849 - accuracy: 0.9705 - val_loss: 0.0502 - val_accuracy: 0.9821
Epoch 

In [17]:
pred=model.predict(X_test)

27/27 [==============================] - 0s 3ms/step


In [18]:
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [19]:
accuracy_score(y_test,pred)

0.9809523809523809

**Using Naive Bayes**

In [20]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(X_train, y_train)

pred_gnb = gnb.predict(X_test)

**Support Vector Machine**

In [21]:
# SVM
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)
pred_svm=clf.predict(X_test)

**K Nearest Neighbor**

In [23]:

from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

pred_knn = neigh.predict(X_test)

**Decision tree**

In [24]:
from sklearn import tree

dt = tree.DecisionTreeClassifier()
dt = dt.fit(X_train, y_train)

pred_dt = dt.predict(X_test)

**CNN**

In [25]:
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

In [39]:
X_train.shape

(3360, 4717)

In [32]:
model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, input_shape=(64,64,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation='sigmoid')
])




In [56]:
model = models.Sequential()
model.add(layers.Conv1D(32, 1, activation = 'relu', input_shape = (1,4717)))
model.add(layers.Conv1D(32, 1, activation = 'relu'))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation = 'sigmoid'))
model.summary()
model.compile(optimizer = 'adam', loss = tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])

X_train1 = X_train.reshape(-1, 1, 4717)
X_test1 = X_test.reshape(-1, 1, 4717)

history = model.fit(X_train1, y_train, epochs = 10, validation_data = (X_test1, y_test))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 1, 32)             150976    
                                                                 
 conv1d_3 (Conv1D)           (None, 1, 32)             1056      
                                                                 
 flatten_5 (Flatten)         (None, 32)                0         
                                                                 
 dense_21 (Dense)            (None, 1)                 33        
                                                                 
Total params: 152,065
Trainable params: 152,065
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
105/105 [==============================] - 1s 9ms/step - loss: 0.4161 - accuracy: 0.8333 - val_loss: 0.2034 - val_accuracy: 0.8905
Epoch 2/10
105/105 [==================

In [57]:
y_pred = model.predict(X_test1).flatten()
# y_pred1 = [1 if x>-0.5 else 0 for x in y_pred]
y_pred = np.round(y_pred)

27/27 [==============================] - 0s 3ms/step


In [58]:

accuracy_score(y_test,pred)

0.9809523809523809

**Saving Model**

In [59]:
from keras.models import load_model
import pickle

model.save('/content/SQLInjection/my_model_cnn1.h5')
with open('vectorizer_cnn', 'wb') as fin:
    pickle.dump(vectorizer, fin)

**Confusion Matrix**

In [60]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy
def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision
def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall
def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

**Compare between methods**

In [47]:
#CNN
accuracy,precision,recall=confusion_matrix(y_test,pred)
print(" For CNN \n Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For CNN 
 Accuracy : 0.9809523809523809 
 Precision : 0.9451476793248945 
 Recall : 0.986784140969163


In [48]:
#Naive Bayes
accuracy,precision,recall=confusion_matrix(y_test,pred_gnb)
print(" For Naive Bayes Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For Naive Bayes Accuracy : 0.9833333333333333 
 Precision : 0.941908713692946 
 Recall : 1.0


In [49]:
#Support vector machine
accuracy,precision,recall=confusion_matrix(y_test,pred_svm)
print(" For SVM Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For SVM Accuracy : 0.7869047619047619 
 Precision : 1.0 
 Recall : 0.21145374449339208


In [50]:
#K Nearest neighbor
accuracy,precision,recall=confusion_matrix(y_test,pred_knn)
print(" For KNN Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For KNN Accuracy : 0.5726190476190476 
 Precision : 0.3873720136518771 
 Recall : 1.0


In [51]:
#Decession tree
accuracy,precision,recall=confusion_matrix(y_test,pred_dt)
print(" For Decision Tree Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 For Decision Tree Accuracy : 0.888095238095238 
 Precision : 0.7071651090342679 
 Recall : 1.0


# **Classifying user data**

In [69]:
import keras
from keras.models import load_model
import pickle


mymodel = tf.keras.models.load_model('/content/SQLInjection/my_model_cnn1.h5')
myvectorizer = pickle.load(open("vectorizer_cnn", 'rb'))




def clean_data(input_val):

    input_val=input_val.replace('\n', '')
    input_val=input_val.replace('%20', ' ')
    input_val=input_val.replace('=', ' = ')
    input_val=input_val.replace('((', ' (( ')
    input_val=input_val.replace('))', ' )) ')
    input_val=input_val.replace('(', ' ( ')
    input_val=input_val.replace(')', ' ) ')
    input_val=input_val.replace('1 ', 'numeric')
    input_val=input_val.replace(' 1', 'numeric')
    input_val=input_val.replace("'1 ", "'numeric ")
    input_val=input_val.replace(" 1'", " numeric'")
    input_val=input_val.replace('1,', 'numeric,')
    input_val=input_val.replace(" 2 ", " numeric ")
    input_val=input_val.replace(' 3 ', ' numeric ')
    input_val=input_val.replace(' 3--', ' numeric--')
    input_val=input_val.replace(" 4 ", ' numeric ')
    input_val=input_val.replace(" 5 ", ' numeric ')
    input_val=input_val.replace(' 6 ', ' numeric ')
    input_val=input_val.replace(" 7 ", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace('1234', ' numeric ')
    input_val=input_val.replace("22", ' numeric ')
    input_val=input_val.replace(" 8 ", ' numeric ')
    input_val=input_val.replace(" 200 ", ' numeric ')
    input_val=input_val.replace("23 ", ' numeric ')
    input_val=input_val.replace('"1', '"numeric')
    input_val=input_val.replace('1"', '"numeric')
    input_val=input_val.replace("7659", 'numeric')
    input_val=input_val.replace(" 37 ", ' numeric ')
    input_val=input_val.replace(" 45 ", ' numeric ')

    return input_val








def predict_sqli_attack():
    
    repeat=True
    
    beautify=''
    for i in range(20):
        beautify+= "="

    print(beautify) 
    input_val=input("Give me some data to work on : ")
    print(beautify)

    
    if input_val== '0':
        repeat=False
    
    

    input_val=clean_data(input_val)
    input_val=[input_val]



    input_val=myvectorizer.transform(input_val).toarray()
    
    input_val.shape=(-1,1,4717)

    result=mymodel.predict(input_val)


    print(beautify)
    
    
    if repeat == True:
        
        if result>0.5:
            print("ALERT :::: This can be SQL injection")


        elif result<=0.5:
            print("It seems to be safe")
            
        print(beautify)
            
        predict_sqli_attack()
            
    elif repeat == False:
        print( " Good Bye ")


In [70]:
predict_sqli_attack()

Give me some data to work on : 1=1
1/1 [==============================] - 0s 72ms/step
ALERT :::: This can be SQL injection
Give me some data to work on : hoang hoa
1/1 [==============================] - 0s 17ms/step
ALERT :::: This can be SQL injection
Give me some data to work on : 3123
1/1 [==============================] - 0s 17ms/step
ALERT :::: This can be SQL injection
Give me some data to work on : 1
1/1 [==============================] - 0s 17ms/step
ALERT :::: This can be SQL injection
Give me some data to work on : o
1/1 [==============================] - 0s 24ms/step
ALERT :::: This can be SQL injection
Give me some data to work on : yeeu em
1/1 [==============================] - 0s 17ms/step
ALERT :::: This can be SQL injection
Give me some data to work on : wait for me
1/1 [==============================] - 0s 19ms/step
It seems to be safe
Give me some data to work on : last round
1/1 [==============================] - 0s 16ms/step
It seems to be safe
Give me some data to